In [ ]:
"""
Workflow:
1. Download the YouTube audio file.
2. Transcribe the audio using Whisper.
3. Summarize the transcribed text using LangChain with three different approaches: stuff, refine, and map_reduce.
4. Adding multiple URLs to DeepLake database, and retrieving information. 
"""

In [2]:
"""Packages to be used:
yt_dlp
whisper
langchian
deeplake
langchain_generative_ai
tiktoken
ffmpeg"""

'Packages to be used:\nyt_dlp\nwhisper\nlangchian\ndeeplake\nlangchain_generative_ai\ntiktoken'

# Function to download youtube videos

In [4]:
import yt_dlp

def download_mp4_from_youtube(url, file_name='lecuninterview.mp4'):
    #setting options for download
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': file_name,
        'quiet': True,
    }

    #downloading video
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)


url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error


DownloadError: ERROR: You have requested merging of multiple formats but ffmpeg is not installed. Aborting due to --abort-on-error